In [1]:
import re
import random
from ast import literal_eval
from collections import Counter
import fasttext
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from skmultilearn.model_selection import iterative_train_test_split
from skmultilearn.model_selection.measures import get_combination_wise_output_matrix

In [2]:
tqdm.pandas()

/home/abdullah/anaconda3/lib/python3.8/site-packages/tqdm/std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [3]:
documents = pd.read_csv("final_before_split/immap_dfs_documents.csv")
sentences = pd.read_csv("final_before_split/immap_dfs_sentences.csv")

In [4]:
sentences["sector_ids"] = sentences["sector_ids"].progress_apply(literal_eval)

In [5]:
sec_ids = set()
for sec_ids_list in sentences["sector_ids"]:
    sec_ids.update(sec_ids_list)
sec_ids

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10}

In [6]:
sentences["sec_0"] = 0
sentences["sec_1"] = 0
sentences["sec_2"] = 0
sentences["sec_3"] = 0
sentences["sec_4"] = 0
sentences["sec_6"] = 0
sentences["sec_7"] = 0
sentences["sec_8"] = 0
sentences["sec_9"] = 0
sentences["sec_10"] = 0

In [7]:
for row in tqdm(sentences.iterrows(), total=sentences.shape[0]):
    sec_ids = row[1]["sector_ids"]
    idx = row[0]
    if 0 in sec_ids:
        sentences.loc[idx, "sec_0"] = 1
    if 1 in sec_ids:
        sentences.loc[idx, "sec_1"] = 1
    if 2 in sec_ids:
        sentences.loc[idx, "sec_2"] = 1
    if 3 in sec_ids:
        sentences.loc[idx, "sec_3"] = 1
    if 4 in sec_ids:
        sentences.loc[idx, "sec_4"] = 1
    if 5 in sec_ids:
        sentences.loc[idx, "sec_5"] = 1
    if 6 in sec_ids:
        sentences.loc[idx, "sec_6"] = 1
    if 7 in sec_ids:
        sentences.loc[idx, "sec_7"] = 1
    if 8 in sec_ids:
        sentences.loc[idx, "sec_8"] = 1
    if 9 in sec_ids:
        sentences.loc[idx, "sec_9"] = 1
    if 10 in sec_ids:
        sentences.loc[idx, "sec_10"] = 1     

In [40]:
sentences.sum(numeric_only=True).astype(np.int)

doc_id         15916292190
sentence_id      340022757
is_relevant          67945
sec_0                 2134
sec_1                 6175
sec_2                 4033
sec_3                 6528
sec_4                17287
sec_6                 1820
sec_7                 1959
sec_8                 8813
sec_9                 3131
sec_10                4453
sec_5                 6922
dtype: int64

In [10]:
len(sentences)

355715

In [11]:
sentences["lang_code"].value_counts()

en    236961
es     59590
fr     59164
Name: lang_code, dtype: int64

In [12]:
sentences_en = sentences[sentences["lang_code"].eq("en")]
sentences_fr = sentences[sentences["lang_code"].eq("fr")]
sentences_es = sentences[sentences["lang_code"].eq("es")]

In [36]:
def transform_labels(df):
    labels = np.zeros([len(df), 11])
    for i, sec_ids in enumerate(df["sector_ids"]):
        for sec_id in sec_ids:
            labels[i, sec_id] = 1
    return labels

In [13]:
def train_val_test_df_split(df):
    labels = np.zeros([len(df), 11])
    for i, sec_ids in enumerate(df["sector_ids"]):
        for sec_id in sec_ids:
            labels[i, sec_id] = 1
    X_train, y_train, X_test, y_test = iterative_train_test_split(df["sentence_text"].to_numpy().reshape(-1, 1), labels, test_size = 0.1)
    X_train, y_train, X_val, y_val = iterative_train_test_split(X_train, y_train, test_size = 0.1)
    return {
        "X_train": X_train, "y_train": y_train, "X_test": X_test, "y_test": y_test, "X_val": X_val, "y_val": y_val
    }

In [14]:
X_train_en, y_train_en, X_test_en, y_test_en, X_val_en, y_val_en = list(train_val_test_df_split(sentences_en).values())

In [15]:
X_train_fr, y_train_fr, X_test_fr, y_test_fr, X_val_fr, y_val_fr = list(train_val_test_df_split(sentences_fr).values())

In [16]:
X_train_es, y_train_es, X_test_es, y_test_es, X_val_es, y_val_es = list(train_val_test_df_split(sentences_es).values())

In [35]:
sentences_en.sum(numeric_only=True).astype(np.int)

doc_id         10420874061
sentence_id      295368456
is_relevant          41622
sec_0                  918
sec_1                 4557
sec_2                 2291
sec_3                 3926
sec_4                12597
sec_6                 1393
sec_7                 1151
sec_8                 4233
sec_9                 1944
sec_10                2915
sec_5                 3559
dtype: int64

In [41]:
sentences_en.sum(numeric_only=True).astype(np.int)/sentences_en.shape[0]

doc_id         43977.169496
sentence_id     1246.485523
is_relevant        0.175649
sec_0              0.003874
sec_1              0.019231
sec_2              0.009668
sec_3              0.016568
sec_4              0.053161
sec_6              0.005879
sec_7              0.004857
sec_8              0.017864
sec_9              0.008204
sec_10             0.012302
sec_5              0.015019
dtype: float64

In [38]:
en_labels = transform_labels(sentences_en)
y_train_en.sum(axis=0)/en_labels.sum(axis=0) ,y_val_en.sum(axis=0)/en_labels.sum(axis=0), y_test_en.sum(axis=0)/en_labels.sum(axis=0)

(array([0.80936819, 0.80996269, 0.81012658, 0.80998472, 0.80995475,
        0.81005901, 0.81048098, 0.80973067, 0.81006378, 0.81018519,
        0.80994854]),
 array([0.09041394, 0.08997147, 0.08991707, 0.0899134 , 0.09002143,
        0.0899129 , 0.08973439, 0.09035621, 0.09000709, 0.09002058,
        0.08987993]),
 array([0.10021786, 0.10006583, 0.09995635, 0.10010188, 0.10002382,
        0.1000281 , 0.09978464, 0.09991312, 0.09992913, 0.09979424,
        0.10017153]))

In [20]:
sentences_en_train = sentences_en[sentences_en["sentence_text"].isin(X_train_en.reshape(-1))]
sentences_en_val = sentences_en[sentences_en["sentence_text"].isin(X_val_en.reshape(-1))]
sentences_en_test = sentences_en[sentences_en["sentence_text"].isin(X_test_en.reshape(-1))]

In [22]:
sentences_en_train.index & sentences_en_val.index, sentences_en_train.index & sentences_en_test.index, sentences_en_test.index & sentences_en_val.index 

(Int64Index([], dtype='int64'),
 Int64Index([], dtype='int64'),
 Int64Index([], dtype='int64'))

In [23]:
sentences_fr_train = sentences_fr[sentences_fr["sentence_text"].isin(X_train_fr.reshape(-1))]
sentences_fr_val = sentences_fr[sentences_fr["sentence_text"].isin(X_val_fr.reshape(-1))]
sentences_fr_test = sentences_fr[sentences_fr["sentence_text"].isin(X_test_fr.reshape(-1))]
##
sentences_fr_train.index & sentences_fr_val.index, sentences_fr_train.index & sentences_fr_test.index, sentences_fr_test.index & sentences_fr_val.index 

(Int64Index([], dtype='int64'),
 Int64Index([], dtype='int64'),
 Int64Index([], dtype='int64'))

In [24]:
sentences_es_train = sentences_es[sentences_es["sentence_text"].isin(X_train_es.reshape(-1))]
sentences_es_val = sentences_es[sentences_es["sentence_text"].isin(X_val_es.reshape(-1))]
sentences_es_test = sentences_es[sentences_es["sentence_text"].isin(X_test_es.reshape(-1))]
##
sentences_es_train.index & sentences_es_val.index, sentences_es_train.index & sentences_es_test.index, sentences_es_test.index & sentences_es_val.index 

(Int64Index([], dtype='int64'),
 Int64Index([], dtype='int64'),
 Int64Index([], dtype='int64'))

In [29]:
sentences_en_train[['doc_id', 'sentence_id', 'sentence_text', 'is_relevant', 'sector_ids']].to_csv("final_data/en/sentences_en_train.csv", index=None)
sentences_en_val[['doc_id', 'sentence_id', 'sentence_text', 'is_relevant', 'sector_ids']].to_csv("final_data/en/sentences_en_val.csv", index=None)
sentences_en_test[['doc_id', 'sentence_id', 'sentence_text', 'is_relevant', 'sector_ids']].to_csv("final_data/en/sentences_en_test.csv", index=None)
sentences_en_test[['doc_id', 'sentence_id', 'sentence_text']].to_csv("final_data/en/sentences_en_test_without_ground_truth.csv", index=None)

In [32]:
sentences_en_train.to_csv("my_final_data/en/sentences_en_train.csv", index=None)
sentences_en_val.to_csv("my_final_data/en/sentences_en_val.csv", index=None)
sentences_en_test.to_csv("my_final_data/en/sentences_en_test.csv", index=None)

In [30]:
sentences_fr_train[['doc_id', 'sentence_id', 'sentence_text', 'is_relevant', 'sector_ids']].to_csv("final_data/fr/sentences_fr_train.csv", index=None)
sentences_fr_val[['doc_id', 'sentence_id', 'sentence_text', 'is_relevant', 'sector_ids']].to_csv("final_data/fr/sentences_fr_val.csv", index=None)
sentences_fr_test[['doc_id', 'sentence_id', 'sentence_text', 'is_relevant', 'sector_ids']].to_csv("final_data/fr/sentences_fr_test.csv", index=None)
sentences_fr_test[['doc_id', 'sentence_id', 'sentence_text']].to_csv("final_data/fr/sentences_fr_test_without_ground_truth.csv", index=None)

In [33]:
sentences_fr_train.to_csv("my_final_data/fr/sentences_fr_train.csv", index=None)
sentences_fr_val.to_csv("my_final_data/fr/sentences_fr_val.csv", index=None)
sentences_fr_test.to_csv("my_final_data/fr/sentences_fr_test.csv", index=None)

In [31]:
sentences_es_train[['doc_id', 'sentence_id', 'sentence_text', 'is_relevant', 'sector_ids']].to_csv("final_data/es/sentences_es_train.csv", index=None)
sentences_es_val[['doc_id', 'sentence_id', 'sentence_text', 'is_relevant', 'sector_ids']].to_csv("final_data/es/sentences_es_val.csv", index=None)
sentences_es_test[['doc_id', 'sentence_id', 'sentence_text', 'is_relevant', 'sector_ids']].to_csv("final_data/es/sentences_es_test.csv", index=None)
sentences_es_test[['doc_id', 'sentence_id', 'sentence_text']].to_csv("final_data/es/sentences_es_test_without_ground_truth.csv", index=None)

In [34]:
sentences_es_train.to_csv("my_final_data/es/sentences_es_train.csv", index=None)
sentences_es_val.to_csv("my_final_data/es/sentences_es_val.csv", index=None)
sentences_es_test.to_csv("my_final_data/es/sentences_es_test.csv", index=None)